# SME Market Segmentation  
# Part 1 - Preparing Data

*By Ricky Soo*  
*19th November 2020*  
*https://github.com/rickysoo*

How can a bank serve its customers better? How can it better position itself to provide loans to the small and medium sized enterprises (SMEs) to help them in their business?

Given 3 sets of data - SME bank accounts information, bank transactions in 2018, and bank transactions in 2019, let's segment the SME market into different clusters to gain insights into their individual characteristics and behaviors.

In this study, the usual data science process is followed to gain insights into the available data. The k-means algorithm is used to cluster the existing 112,618 customer accounts into 6 clusters. Insights are gained from each clusters. Then a discussion follows on how to better serve the 6 segments, as well as targeting the non-customers out there who are not in the data.

Part 1 - Preparing Data  
Part 2 - Exploring Data  
Part 3 - Feature Engineering  
Part 4 - Model Training
Part 5 - Discussion

The conclusion is, compete for today and compete for tomorrow. Go above market segmentation. Be an entrepreneur bank.

## 1.1 Preparing Data - SME Info

In this section, data on SME accounts are loaded, explored and cleaned.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Loading Data

In [2]:
df_sme_info = pd.read_csv('sme_info.csv')
df_sme_info.head()

,Customer Account ID,Customer Company ID,Borrowing/Non-Borrowing Customer,Asset/Liability,Product Category,Operating Years,Industry Sector,Constitution_description,Islamic_Tagging (Yes/No),State,Product Type,Tenor
0,Cust_Acct_ID_000001,Cust_Party_ID_122133,Non-borrowing,Liability,Deposits,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,Cust_Acct_ID_000002,Cust_Party_ID_059101,Non-borrowing,Liability,Deposits,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,Cust_Acct_ID_000003,Cust_Party_ID_081657,Non-borrowing,Liability,Deposits,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,Cust_Acct_ID_000004,Cust_Party_ID_010950,Non-borrowing,Liability,Deposits,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,Cust_Acct_ID_000005,Cust_Party_ID_092549,Non-borrowing,Liability,Deposits,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


In [3]:
df_sme_info.shape

(141749, 12)

In [4]:
df_sme_info.columns

Index(['Customer Account ID', 'Customer Company ID',
       'Borrowing/Non-Borrowing Customer', 'Asset/Liability',
       'Product Category', 'Operating Years', 'Industry Sector',
       'Constitution_description', 'Islamic_Tagging (Yes/No)', 'State',
       'Product Type', 'Tenor'],
      dtype='object')

Change the column names for easier data wrangling.

In [5]:
df_sme_info.columns = ['AccountID', 'CompanyID', 'Borrowing', 'Asset', 'Product', 'Years', 'Industry', 'ConstType', 'Islamic', 'State', 'ProductType', 'Tenor']

In [6]:
df_sme_info.columns

Index(['AccountID', 'CompanyID', 'Borrowing', 'Asset', 'Product', 'Years',
       'Industry', 'ConstType', 'Islamic', 'State', 'ProductType', 'Tenor'],
      dtype='object')

In [7]:
df_sme_info.head()

,AccountID,CompanyID,Borrowing,Asset,Product,Years,Industry,ConstType,Islamic,State,ProductType,Tenor
0,Cust_Acct_ID_000001,Cust_Party_ID_122133,Non-borrowing,Liability,Deposits,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,Cust_Acct_ID_000002,Cust_Party_ID_059101,Non-borrowing,Liability,Deposits,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,Cust_Acct_ID_000003,Cust_Party_ID_081657,Non-borrowing,Liability,Deposits,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,Cust_Acct_ID_000004,Cust_Party_ID_010950,Non-borrowing,Liability,Deposits,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,Cust_Acct_ID_000005,Cust_Party_ID_092549,Non-borrowing,Liability,Deposits,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


### Data Wrangling

Check data for missing values.

In [8]:
df_sme_info.isnull().sum()

AccountID       0
CompanyID      22
Borrowing       0
Asset           0
Product         0
Years          24
Industry        0
ConstType       0
Islamic         0
State           0
ProductType     0
Tenor           0
dtype: int64

**CompanyID** - Drop the Company ID column as we will use the Account ID column for identifying bank accounts and joining tables.

In [9]:
df_sme_info.drop('CompanyID', axis = 1, inplace = True)

**Years** - The operating years info is missing in 24 accounts. Remove the accounts as the number is small and they are missing in random.

In [10]:
df_sme_info.drop(df_sme_info[df_sme_info['Years'].isnull()].index, inplace = True)

In [11]:
df_sme_info.isnull().sum()

AccountID      0
Borrowing      0
Asset          0
Product        0
Years          0
Industry       0
ConstType      0
Islamic        0
State          0
ProductType    0
Tenor          0
dtype: int64

**AccountID** - Slice the Account ID into 6 digits.

In [12]:
df_sme_info['AccountID'] = df_sme_info['AccountID'].str.slice(-6)

In [13]:
df_sme_info.head()

,AccountID,Borrowing,Asset,Product,Years,Industry,ConstType,Islamic,State,ProductType,Tenor
0,000001,Non-borrowing,Liability,Deposits,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,000002,Non-borrowing,Liability,Deposits,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,000003,Non-borrowing,Liability,Deposits,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,000004,Non-borrowing,Liability,Deposits,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,000005,Non-borrowing,Liability,Deposits,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


**Borrowing** - Remove the accounts who are already borrowers. Focus on those who are not yet borrowers.

In [14]:
df_sme_info['Borrowing'].value_counts()

Non-borrowing    137587
Borrowing          4138
Name: Borrowing, dtype: int64

In [15]:
df_sme_info.drop(df_sme_info[df_sme_info['Borrowing'] == 'Borrowing'].index, inplace = True)

In [16]:
df_sme_info.drop(['Borrowing'], axis = 1, inplace = True)

In [17]:
df_sme_info.head()

,AccountID,Asset,Product,Years,Industry,ConstType,Islamic,State,ProductType,Tenor
0,000001,Liability,Deposits,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,000002,Liability,Deposits,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,000003,Liability,Deposits,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,000004,Liability,Deposits,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,000005,Liability,Deposits,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


**Asset** - Drop the column as all of them are liabilities (after previous data cleaning).

In [18]:
df_sme_info['Asset'].value_counts()

Liability    137587
Name: Asset, dtype: int64

In [19]:
df_sme_info.drop(['Asset'], axis = 1, inplace = True)

In [20]:
df_sme_info.head()

,AccountID,Product,Years,Industry,ConstType,Islamic,State,ProductType,Tenor
0,000001,Deposits,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,000002,Deposits,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,000003,Deposits,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,000004,Deposits,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,000005,Deposits,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


**Product** - Drop the column as all of them are deposits (after previous data cleaning).

In [21]:
df_sme_info['Product'].value_counts()

Deposits    137587
Name: Product, dtype: int64

In [22]:
df_sme_info.drop('Product', axis = 1, inplace = True)

In [23]:
df_sme_info.head()

,AccountID,Years,Industry,ConstType,Islamic,State,ProductType,Tenor
0,000001,19.0,Wholesale & Retail Trade,PARTNERSHIP,N,SABAH,CA,0
1,000002,4.0,Wholesale & Retail Trade,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0
2,000003,4.0,"Transport, Storage & Communication",SOLE PROPRIETOR,N,PERAK,CA,0
3,000004,12.0,Manufacturing,SOLE PROPRIETOR,Y,TERENGGANU,CA,0
4,000005,5.0,Real Estate,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0


**Industry** - Convert into dummy variables.

In [24]:
df_sme_info['Industry'].value_counts()

Wholesale & Retail Trade              50296
Finance/Insurance/Services            25985
Construction                          17946
Education, Health and Others          13317
Manufacturing                         12910
Transport, Storage & Communication     8073
Real Estate                            3245
Agriculture                            3103
Electricity, Gas & Water               1680
Mining & quarrying                      550
Others                                  242
Household                               240
Name: Industry, dtype: int64

In [25]:
df_sme_info['Industry'].replace({
    'Wholesale & Retail Trade': 'Trade',
    'Finance/Insurance/Services': 'Finance',
    'Construction': 'Construction',
    'Manufacturing': 'Manufacturing',
    'Education, Health and Others': 'EducationHealth',
    'Transport, Storage & Communication': 'Logistics',
    'Real Estate': 'RealEstate',
    'Agriculture': 'Agriculture',
    'Electricity, Gas & Water': 'Utilities',
    'Mining & quarrying': 'Mining',
    'Household': 'Household',
    'Others': 'Others'
}, inplace = True)

In [26]:
df_sme_info = pd.get_dummies(df_sme_info, columns = ['Industry'])

In [27]:
df_sme_info.head()

,AccountID,Years,ConstType,Islamic,State,ProductType,Tenor,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,Industry_Logistics,Industry_Manufacturing,Industry_Mining,Industry_Others,Industry_RealEstate,Industry_Trade,Industry_Utilities
0,000001,19.0,PARTNERSHIP,N,SABAH,CA,0,0,0,0,0,0,0,0,0,0,0,1,0
1,000002,4.0,SOLE PROPRIETOR,Y,KUALA LUMPUR,CA,0,0,0,0,0,0,0,0,0,0,0,1,0
2,000003,4.0,SOLE PROPRIETOR,N,PERAK,CA,0,0,0,0,0,0,1,0,0,0,0,0,0
3,000004,12.0,SOLE PROPRIETOR,Y,TERENGGANU,CA,0,0,0,0,0,0,0,1,0,0,0,0,0
4,000005,5.0,SDN BHD/PRIVATE LTD,N,KUALA LUMPUR,CA,0,0,0,0,0,0,0,0,0,0,1,0,0


**ConstType** - Convert into dummy variables. Drop a number of accounts which are not SMES or are small in numbers.

In [28]:
df_sme_info['ConstType'].value_counts()

SOLE PROPRIETOR         72456
SDN BHD/PRIVATE LTD     41721
PARTNERSHIP             21239
ASSOC/SCHOOL/SOCIETY     1958
BHD/PUBLIC LTD CO          79
OTHERS                     62
COOPERATIVE                37
DNFI-Others                34
MERCHANT BANK               1
Name: ConstType, dtype: int64

In [29]:
df_sme_info['ConstType'].replace({
    'SOLE PROPRIETOR': 'SoleProprietor',
    'SDN BHD/PRIVATE LTD': 'PrivateLtd',
    'PARTNERSHIP': 'Partnership',
    'ASSOC/SCHOOL/SOCIETY': 'SchoolSociety',
    'BHD/PUBLIC LTD CO': 'PublicLtd',
    'OTHERS': 'Others',
    'COOPERATIVE': 'Cooperative',
    'DNFI-Others': 'DNFI',
    'MERCHANT BANK': 'Bank'
}, inplace = True)

In [30]:
df_sme_info.drop(df_sme_info[df_sme_info['ConstType'] == 'Others'].index, inplace = True)

In [31]:
df_sme_info.drop(df_sme_info[df_sme_info['ConstType'] == 'Cooperative'].index, inplace = True)

In [32]:
df_sme_info.drop(df_sme_info[df_sme_info['ConstType'] == 'DNFI'].index, inplace = True)

In [33]:
df_sme_info.drop(df_sme_info[df_sme_info['ConstType'] == 'Bank'].index, inplace = True)

In [34]:
df_sme_info = pd.get_dummies(df_sme_info, columns = ['ConstType'])

In [35]:
df_sme_info.head()

,AccountID,Years,Islamic,State,ProductType,Tenor,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,...,Industry_Mining,Industry_Others,Industry_RealEstate,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor
0,000001,19.0,N,SABAH,CA,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,000002,4.0,Y,KUALA LUMPUR,CA,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,000003,4.0,N,PERAK,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,000004,12.0,Y,TERENGGANU,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,000005,5.0,N,KUALA LUMPUR,CA,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


**Islamic** - Keep the column.

In [36]:
df_sme_info['Islamic'].value_counts()

N    111053
Y     26400
Name: Islamic, dtype: int64

In [37]:
df_sme_info['Islamic'].replace({'N': 0, 'Y': 1}, inplace = True)

In [38]:
df_sme_info['Islamic'].value_counts()

0    111053
1     26400
Name: Islamic, dtype: int64

In [39]:
df_sme_info.head()

,AccountID,Years,Islamic,State,ProductType,Tenor,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,...,Industry_Mining,Industry_Others,Industry_RealEstate,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor
0,000001,19.0,0,SABAH,CA,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,000002,4.0,1,KUALA LUMPUR,CA,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,000003,4.0,0,PERAK,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,000004,12.0,1,TERENGGANU,CA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,000005,5.0,0,KUALA LUMPUR,CA,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


**State** - Remove a number of accounts which are not in Malaysia. Drop the column as it will not be used in model training. Nowadays, geographical location is not as vital as before as banking goes digital.

In [40]:
df_sme_info['State'].value_counts()

SELANGOR        43163
KUALA LUMPUR    24306
JOHOR           17858
PENANG          10303
SARAWAK          9782
PERAK            6598
SABAH            5003
KEDAH            4931
N.SEMBILAN       4306
PAHANG           3915
MALACCA          2521
TERENGGANU       1925
others           1671
KELANTAN          519
PERLIS            358
Foreign           294
Name: State, dtype: int64

In [41]:
df_sme_info.drop(df_sme_info[df_sme_info['State'] == 'Others'].index, inplace = True)

In [42]:
df_sme_info.drop(df_sme_info[df_sme_info['State'] == 'Foreign'].index, inplace = True)

In [43]:
df_sme_info.drop('State', axis = 1, inplace = True)

In [44]:
df_sme_info.head()

,AccountID,Years,Islamic,ProductType,Tenor,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,...,Industry_Mining,Industry_Others,Industry_RealEstate,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor
0,000001,19.0,0,CA,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,000002,4.0,1,CA,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,000003,4.0,0,CA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,000004,12.0,1,CA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,000005,5.0,0,CA,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


**Products** - Convert into dummy variables.

In [45]:
df_sme_info['ProductType'].value_counts()

CA    137138
SA        21
Name: ProductType, dtype: int64

In [46]:
df_sme_info = pd.get_dummies(df_sme_info, columns = ['ProductType'])

In [47]:
df_sme_info.head()

,AccountID,Years,Islamic,Tenor,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,Industry_Logistics,...,Industry_RealEstate,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor,ProductType_CA,ProductType_SA
0,000001,19.0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,000002,4.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,000003,4.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
3,000004,12.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,000005,5.0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0


**Tenor** - Drop the column as it has only one value.

In [48]:
df_sme_info['Tenor'].value_counts()

0    137159
Name: Tenor, dtype: int64

In [49]:
df_sme_info.drop('Tenor', axis = 1, inplace = True)

In [50]:
df_sme_info.head()

,AccountID,Years,Islamic,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,Industry_Logistics,Industry_Manufacturing,...,Industry_RealEstate,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor,ProductType_CA,ProductType_SA
0,000001,19.0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,000002,4.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,000003,4.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
3,000004,12.0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
4,000005,5.0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0


Save the data for future retrieval.

In [51]:
df_sme_info.to_csv('sme_info_cleaned.csv', index = False)

## 1.2 Preparing Data - SME Transactions

In this section, data on 2018 and 2019 are loaded, explored, cleaned and joined to the SME info data.

### 2018 Data

In this section, the 2018 SME transaction data is loaded, explored and cleaned.

In [52]:
df_sme_2018 = pd.read_csv('sme_2018.csv')

In [53]:
df_sme_2018.head()

,Customer Account ID,Customer Company ID,Borrowing/Non-Borrowing Customer,Balance_December,Balance_November,Balance_October,Balance_September,Balance_August,Balance_July,Balance_June,...,Total Debit Count_March,Total Debit Count_April,Total Debit Count_May,Total Debit Count_June,Total Debit Count_July,Total Debit Count_August,Total Debit Count_September,Total Debit Count_October,Total Debit Count_November,Total Debit Count_December
0,Cust_Acct_ID_000001,Cust_Party_ID_122133,Non-borrowing,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,...,22.0,24.0,31.0,19.0,26.0,20.0,16.0,19.0,27.0,16.0
1,Cust_Acct_ID_000002,Cust_Party_ID_059101,Non-borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN
2,Cust_Acct_ID_000004,Cust_Party_ID_010950,Non-borrowing,0.00,51.78,2.72,23.90,23.90,31.90,32.40,...,7.0,1.0,3.0,1.0,3.0,1.0,NaN,14.0,15.0,3.0
3,Cust_Acct_ID_000005,Cust_Party_ID_092549,Non-borrowing,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,...,133.0,108.0,101.0,69.0,87.0,82.0,74.0,61.0,92.0,65.0
4,Cust_Acct_ID_000006,Cust_Party_ID_132914,Non-borrowing,982.50,992.50,992.50,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


In [54]:
df_sme_2018.shape

(118435, 63)

Rename into AccountID column.

In [55]:
df_sme_2018.rename(columns = {
    'Customer Account ID': 'AccountID'
}, inplace = True)

In [56]:
df_sme_2018['AccountID'] = df_sme_2018['AccountID'].str.slice(-6)

In [57]:
df_sme_2018.drop(['Customer Company ID', 'Borrowing/Non-Borrowing Customer'], axis = 1, inplace = True)

In [58]:
df_sme_2018.head()

,AccountID,Balance_December,Balance_November,Balance_October,Balance_September,Balance_August,Balance_July,Balance_June,Balance_May,Balance_April,...,Total Debit Count_March,Total Debit Count_April,Total Debit Count_May,Total Debit Count_June,Total Debit Count_July,Total Debit Count_August,Total Debit Count_September,Total Debit Count_October,Total Debit Count_November,Total Debit Count_December
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,22.0,24.0,31.0,19.0,26.0,20.0,16.0,19.0,27.0,16.0
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,7.0,1.0,3.0,1.0,3.0,1.0,NaN,14.0,15.0,3.0
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,133.0,108.0,101.0,69.0,87.0,82.0,74.0,61.0,92.0,65.0
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


In [59]:
df_sme_2018.shape

(118435, 61)

Check for missing values. Replace with 0 for all the missing balances, amounts and number of transactions.

In [60]:
dict(df_sme_2018.isnull().sum())

{'AccountID': 0,
 'Balance_December': 0,
 'Balance_November': 0,
 'Balance_October': 0,
 'Balance_September': 0,
 'Balance_August': 0,
 'Balance_July': 0,
 'Balance_June': 0,
 'Balance_May': 0,
 'Balance_April': 0,
 'Balance_March': 0,
 'Balance_February': 0,
 'Balance_January': 0,
 'Total Credit Amount_January': 49267,
 'Total Credit Amount_February': 49863,
 'Total Credit Amount_March': 48359,
 'Total Credit Amount_April': 47049,
 'Total Credit Amount_May': 46770,
 'Total Credit Amount_June': 46157,
 'Total Credit Amount_July': 36211,
 'Total Credit Amount_August': 33271,
 'Total Credit Amount_September': 43692,
 'Total Credit Amount_October': 40483,
 'Total Credit Amount_November': 40721,
 'Total Credit Amount_December': 39863,
 'Total Debit Amount_January': 48893,
 'Total Debit Amount_February': 49575,
 'Total Debit Amount_March': 47472,
 'Total Debit Amount_April': 46496,
 'Total Debit Amount_May': 46117,
 'Total Debit Amount_June': 25306,
 'Total Debit Amount_July': 35823,
 'Tota

In [61]:
df_sme_2018.fillna(0, inplace = True)

In [62]:
dict(df_sme_2018.isnull().sum())

{'AccountID': 0,
 'Balance_December': 0,
 'Balance_November': 0,
 'Balance_October': 0,
 'Balance_September': 0,
 'Balance_August': 0,
 'Balance_July': 0,
 'Balance_June': 0,
 'Balance_May': 0,
 'Balance_April': 0,
 'Balance_March': 0,
 'Balance_February': 0,
 'Balance_January': 0,
 'Total Credit Amount_January': 0,
 'Total Credit Amount_February': 0,
 'Total Credit Amount_March': 0,
 'Total Credit Amount_April': 0,
 'Total Credit Amount_May': 0,
 'Total Credit Amount_June': 0,
 'Total Credit Amount_July': 0,
 'Total Credit Amount_August': 0,
 'Total Credit Amount_September': 0,
 'Total Credit Amount_October': 0,
 'Total Credit Amount_November': 0,
 'Total Credit Amount_December': 0,
 'Total Debit Amount_January': 0,
 'Total Debit Amount_February': 0,
 'Total Debit Amount_March': 0,
 'Total Debit Amount_April': 0,
 'Total Debit Amount_May': 0,
 'Total Debit Amount_June': 0,
 'Total Debit Amount_July': 0,
 'Total Debit Amount_August': 0,
 'Total Debit Amount_September': 0,
 'Total Debit

In [63]:
df_sme_2018.head()

,AccountID,Balance_December,Balance_November,Balance_October,Balance_September,Balance_August,Balance_July,Balance_June,Balance_May,Balance_April,...,Total Debit Count_March,Total Debit Count_April,Total Debit Count_May,Total Debit Count_June,Total Debit Count_July,Total Debit Count_August,Total Debit Count_September,Total Debit Count_October,Total Debit Count_November,Total Debit Count_December
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,22.0,24.0,31.0,19.0,26.0,20.0,16.0,19.0,27.0,16.0
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,7.0,1.0,3.0,1.0,3.0,1.0,0.0,14.0,15.0,3.0
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,133.0,108.0,101.0,69.0,87.0,82.0,74.0,61.0,92.0,65.0
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


### 2019 Data

In this section, the 2019 SME transaction data is loaded, explored and cleaned.

In [64]:
df_sme_2019 = pd.read_csv('sme_2019.csv')

In [65]:
df_sme_2019.head()

,Customer Account ID,Customer Company ID,Borrowing/Non-Borrowing Customer,Balance_December,Balance_November,Balance_October,Balance_September,Balance_August,Balance_July,Balance_June,...,Total Debit Count_March,Total Debit Count_April,Total Debit Count_May,Total Debit Count_June,Total Debit Count_July,Total Debit Count_August,Total Debit Count_September,Total Debit Count_October,Total Debit Count_November,Total Debit Count_December
0,Cust_Acct_ID_000001,Cust_Party_ID_122133,Non-borrowing,25151.78,4770.81,22164.26,14402.67,22927.88,20341.49,28378.77,...,27.0,15.0,19.0,15.0,26.0,15.0,17.0,13.0,15.0,13.0
1,Cust_Acct_ID_000002,Cust_Party_ID_059101,Non-borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cust_Acct_ID_000003,Cust_Party_ID_081657,Non-borrowing,42.32,3.38,7.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,17.0,9.0
3,Cust_Acct_ID_000004,Cust_Party_ID_010950,Non-borrowing,70.00,2.00,2.00,2.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,6.0
4,Cust_Acct_ID_000005,Cust_Party_ID_092549,Non-borrowing,246266.82,229649.52,172551.00,110279.28,231150.88,242288.65,203522.05,...,63.0,71.0,73.0,46.0,66.0,76.0,83.0,56.0,74.0,72.0


In [66]:
df_sme_2019.shape

(140894, 63)

Rename into AccountID column.

In [67]:
df_sme_2019.rename(columns = {
    'Customer Account ID': 'AccountID'
}, inplace = True)

In [68]:
df_sme_2019['AccountID'] = df_sme_2019['AccountID'].str.slice(-6)

In [69]:
df_sme_2019.drop(['Customer Company ID', 'Borrowing/Non-Borrowing Customer'], axis = 1, inplace = True)

In [70]:
df_sme_2019.head()

,AccountID,Balance_December,Balance_November,Balance_October,Balance_September,Balance_August,Balance_July,Balance_June,Balance_May,Balance_April,...,Total Debit Count_March,Total Debit Count_April,Total Debit Count_May,Total Debit Count_June,Total Debit Count_July,Total Debit Count_August,Total Debit Count_September,Total Debit Count_October,Total Debit Count_November,Total Debit Count_December
0,000001,25151.78,4770.81,22164.26,14402.67,22927.88,20341.49,28378.77,30688.41,37063.95,...,27.0,15.0,19.0,15.0,26.0,15.0,17.0,13.0,15.0,13.0
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000003,42.32,3.38,7.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,17.0,9.0
3,000004,70.00,2.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,6.0
4,000005,246266.82,229649.52,172551.00,110279.28,231150.88,242288.65,203522.05,227665.15,251274.15,...,63.0,71.0,73.0,46.0,66.0,76.0,83.0,56.0,74.0,72.0


In [71]:
df_sme_2019.shape

(140894, 61)

Check for missing values. Replace with 0 for all the missing balances, amounts and number of transactions.

In [72]:
dict(df_sme_2019.isnull().sum())

{'AccountID': 0,
 'Balance_December': 0,
 'Balance_November': 0,
 'Balance_October': 0,
 'Balance_September': 0,
 'Balance_August': 0,
 'Balance_July': 0,
 'Balance_June': 0,
 'Balance_May': 0,
 'Balance_April': 0,
 'Balance_March': 0,
 'Balance_February': 0,
 'Balance_January': 0,
 'Total Credit Amount_January': 59587,
 'Total Credit Amount_February': 63455,
 'Total Credit Amount_March': 59377,
 'Total Credit Amount_April': 57506,
 'Total Credit Amount_May': 56376,
 'Total Credit Amount_June': 61484,
 'Total Credit Amount_July': 57154,
 'Total Credit Amount_August': 56951,
 'Total Credit Amount_September': 57056,
 'Total Credit Amount_October': 54630,
 'Total Credit Amount_November': 54659,
 'Total Credit Amount_December': 53796,
 'Total Debit Amount_January': 59542,
 'Total Debit Amount_February': 62326,
 'Total Debit Amount_March': 58815,
 'Total Debit Amount_April': 57297,
 'Total Debit Amount_May': 56040,
 'Total Debit Amount_June': 32485,
 'Total Debit Amount_July': 54434,
 'Tota

In [73]:
df_sme_2019.fillna(0, inplace = True)

In [74]:
dict(df_sme_2019.isnull().sum())

{'AccountID': 0,
 'Balance_December': 0,
 'Balance_November': 0,
 'Balance_October': 0,
 'Balance_September': 0,
 'Balance_August': 0,
 'Balance_July': 0,
 'Balance_June': 0,
 'Balance_May': 0,
 'Balance_April': 0,
 'Balance_March': 0,
 'Balance_February': 0,
 'Balance_January': 0,
 'Total Credit Amount_January': 0,
 'Total Credit Amount_February': 0,
 'Total Credit Amount_March': 0,
 'Total Credit Amount_April': 0,
 'Total Credit Amount_May': 0,
 'Total Credit Amount_June': 0,
 'Total Credit Amount_July': 0,
 'Total Credit Amount_August': 0,
 'Total Credit Amount_September': 0,
 'Total Credit Amount_October': 0,
 'Total Credit Amount_November': 0,
 'Total Credit Amount_December': 0,
 'Total Debit Amount_January': 0,
 'Total Debit Amount_February': 0,
 'Total Debit Amount_March': 0,
 'Total Debit Amount_April': 0,
 'Total Debit Amount_May': 0,
 'Total Debit Amount_June': 0,
 'Total Debit Amount_July': 0,
 'Total Debit Amount_August': 0,
 'Total Debit Amount_September': 0,
 'Total Debit

### Combining 2018 and 2019 Data

In this section, both the 2018 and 2019 data are combined into one before merging with the SME accounts data.

Inner join is used to discard accounts are are missing in either one of the two years.

In [75]:
df_sme_2018_2019 = df_sme_2018.merge(df_sme_2019, how = 'inner', on = 'AccountID', suffixes = ['_2018', '_2019'])

In [76]:
df_sme_2018_2019.head()

,AccountID,Balance_December_2018,Balance_November_2018,Balance_October_2018,Balance_September_2018,Balance_August_2018,Balance_July_2018,Balance_June_2018,Balance_May_2018,Balance_April_2018,...,Total Debit Count_March_2019,Total Debit Count_April_2019,Total Debit Count_May_2019,Total Debit Count_June_2019,Total Debit Count_July_2019,Total Debit Count_August_2019,Total Debit Count_September_2019,Total Debit Count_October_2019,Total Debit Count_November_2019,Total Debit Count_December_2019
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,27.0,15.0,19.0,15.0,26.0,15.0,17.0,13.0,15.0,13.0
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,6.0
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,63.0,71.0,73.0,46.0,66.0,76.0,83.0,56.0,74.0,72.0
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,4.0,3.0,1.0,2.0,2.0,2.0,4.0


In [77]:
df_sme_2018_2019.shape

(118431, 121)

In [78]:
list(df_sme_2018_2019.columns)

['AccountID',
 'Balance_December_2018',
 'Balance_November_2018',
 'Balance_October_2018',
 'Balance_September_2018',
 'Balance_August_2018',
 'Balance_July_2018',
 'Balance_June_2018',
 'Balance_May_2018',
 'Balance_April_2018',
 'Balance_March_2018',
 'Balance_February_2018',
 'Balance_January_2018',
 'Total Credit Amount_January_2018',
 'Total Credit Amount_February_2018',
 'Total Credit Amount_March_2018',
 'Total Credit Amount_April_2018',
 'Total Credit Amount_May_2018',
 'Total Credit Amount_June_2018',
 'Total Credit Amount_July_2018',
 'Total Credit Amount_August_2018',
 'Total Credit Amount_September_2018',
 'Total Credit Amount_October_2018',
 'Total Credit Amount_November_2018',
 'Total Credit Amount_December_2018',
 'Total Debit Amount_January_2018',
 'Total Debit Amount_February_2018',
 'Total Debit Amount_March_2018',
 'Total Debit Amount_April_2018',
 'Total Debit Amount_May_2018',
 'Total Debit Amount_June_2018',
 'Total Debit Amount_July_2018',
 'Total Debit Amount_Au

Extract columns for balances and transactions.

In [79]:
balances = list(df_sme_2018_2019.filter(like = 'Balance').columns)
balances

['Balance_December_2018',
 'Balance_November_2018',
 'Balance_October_2018',
 'Balance_September_2018',
 'Balance_August_2018',
 'Balance_July_2018',
 'Balance_June_2018',
 'Balance_May_2018',
 'Balance_April_2018',
 'Balance_March_2018',
 'Balance_February_2018',
 'Balance_January_2018',
 'Balance_December_2019',
 'Balance_November_2019',
 'Balance_October_2019',
 'Balance_September_2019',
 'Balance_August_2019',
 'Balance_July_2019',
 'Balance_June_2019',
 'Balance_May_2019',
 'Balance_April_2019',
 'Balance_March_2019',
 'Balance_February_2019',
 'Balance_January_2019']

In [80]:
transactions = list(df_sme_2018_2019.filter(like = 'Count').columns)
transactions

['Total Credit Count_January_2018',
 'Total Credit Count_February_2018',
 'Total Credit Count_March_2018',
 'Total Credit Count_April_2018',
 'Total Credit Count_May_2018',
 'Total Credit Count_June_2018',
 'Total Credit Count_July_2018',
 'Total Credit Count_August_2018',
 'Total Credit Count_September_2018',
 'Total Credit Count_October_2018',
 'Total Credit Count_November_2018',
 'Total Credit Count_December_2018',
 'Total Debit Count_January_2018',
 'Total Debit  Count_February_2018',
 'Total Debit  Count_March_2018',
 'Total Debit  Count_April_2018',
 'Total Debit  Count_May_2018',
 'Total Debit  Count_June_2018',
 'Total Debit  Count_July_2018',
 'Total Debit Count_August_2018',
 'Total Debit  Count_September_2018',
 'Total Debit  Count_October_2018',
 'Total Debit  Count_November_2018',
 'Total Debit  Count_December_2018',
 'Total Credit Count_January_2019',
 'Total Credit Count_February_2019',
 'Total Credit Count_March_2019',
 'Total Credit Count_April_2019',
 'Total Credit Co

Create new columns *BalanceMean* for average account balance, *BalanceSD* for standard deviation of monthly balances, and *Transactions* for average number of transactions per month.

In [81]:
df_sme_2018_2019['BalanceMean'] = df_sme_2018_2019[balances].mean(axis = 1)

In [82]:
df_sme_2018_2019['BalanceSD'] = df_sme_2018_2019[balances].std(axis = 1)

In [83]:
df_sme_2018_2019['Transactions'] = df_sme_2018_2019[transactions].sum(axis = 1) / 24

In [84]:
df_sme_2018_2019.head()

,AccountID,Balance_December_2018,Balance_November_2018,Balance_October_2018,Balance_September_2018,Balance_August_2018,Balance_July_2018,Balance_June_2018,Balance_May_2018,Balance_April_2018,...,Total Debit Count_June_2019,Total Debit Count_July_2019,Total Debit Count_August_2019,Total Debit Count_September_2019,Total Debit Count_October_2019,Total Debit Count_November_2019,Total Debit Count_December_2019,BalanceMean,BalanceSD,Transactions
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,15.0,26.0,15.0,17.0,13.0,15.0,13.0,27061.180000,15117.744298,44.625000
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.041667,2.074256,0.208333
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,0.0,0.0,0.0,3.0,0.0,0.0,6.0,62.964167,155.826156,3.791667
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,46.0,66.0,76.0,83.0,56.0,74.0,72.0,328008.590000,111380.697708,138.750000
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,4.0,3.0,1.0,2.0,2.0,2.0,4.0,3843.160417,5122.731455,1.166667


Remove accounts having no transaction at all, and those having zero balance throughout the two years.

In [85]:
df_sme_2018_2019.drop(df_sme_2018_2019[df_sme_2018_2019['Transactions'] == 0].index, inplace = True)
df_sme_2018_2019

,AccountID,Balance_December_2018,Balance_November_2018,Balance_October_2018,Balance_September_2018,Balance_August_2018,Balance_July_2018,Balance_June_2018,Balance_May_2018,Balance_April_2018,...,Total Debit Count_June_2019,Total Debit Count_July_2019,Total Debit Count_August_2019,Total Debit Count_September_2019,Total Debit Count_October_2019,Total Debit Count_November_2019,Total Debit Count_December_2019,BalanceMean,BalanceSD,Transactions
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,15.0,26.0,15.0,17.0,13.0,15.0,13.0,27061.180000,15117.744298,44.625000
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.041667,2.074256,0.208333
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,0.0,0.0,0.0,3.0,0.0,0.0,6.0,62.964167,155.826156,3.791667
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,46.0,66.0,76.0,83.0,56.0,74.0,72.0,328008.590000,111380.697708,138.750000
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,4.0,3.0,1.0,2.0,2.0,2.0,4.0,3843.160417,5122.731455,1.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118426,142292,23497.29,8682.47,46282.56,13795.18,11086.78,4031.27,14313.99,7996.01,27613.49,...,9.0,15.0,17.0,20.0,28.0,20.0,26.0,23233.862083,17544.250744,70.833333
118427,142293,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5530.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5557.343333,31.688589,0.166667
118428,142294,154954.39,118143.27,177612.09,167499.88,67058.13,80188.90,49690.89,58346.21,64483.23,...,78.0,81.0,98.0,89.0,59.0,110.0,97.0,92522.760833,40884.579699,99.000000
118429,142295,44802.68,34973.68,26061.68,23146.68,109953.68,98575.68,96178.18,88229.55,81233.55,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,59856.048750,30704.709009,1.791667


In [86]:
df_sme_2018_2019.drop(df_sme_2018_2019[(df_sme_2018_2019['BalanceMean'] == 0) & (df_sme_2018_2019['BalanceSD'] == 0)].index, inplace = True)
df_sme_2018_2019

,AccountID,Balance_December_2018,Balance_November_2018,Balance_October_2018,Balance_September_2018,Balance_August_2018,Balance_July_2018,Balance_June_2018,Balance_May_2018,Balance_April_2018,...,Total Debit Count_June_2019,Total Debit Count_July_2019,Total Debit Count_August_2019,Total Debit Count_September_2019,Total Debit Count_October_2019,Total Debit Count_November_2019,Total Debit Count_December_2019,BalanceMean,BalanceSD,Transactions
0,000001,10213.52,13661.78,4733.90,13808.25,22176.04,12887.73,34793.38,22606.71,49314.03,...,15.0,26.0,15.0,17.0,13.0,15.0,13.0,27061.180000,15117.744298,44.625000
1,000002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,5.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.041667,2.074256,0.208333
2,000004,0.00,51.78,2.72,23.90,23.90,31.90,32.40,42.40,242.90,...,0.0,0.0,0.0,3.0,0.0,0.0,6.0,62.964167,155.826156,3.791667
3,000005,303275.05,289022.95,377292.55,413555.95,548492.97,498688.25,493999.95,356970.55,420866.65,...,46.0,66.0,76.0,83.0,56.0,74.0,72.0,328008.590000,111380.697708,138.750000
4,000006,982.50,992.50,992.50,0.00,0.00,0.00,0.00,0.00,0.00,...,4.0,3.0,1.0,2.0,2.0,2.0,4.0,3843.160417,5122.731455,1.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118426,142292,23497.29,8682.47,46282.56,13795.18,11086.78,4031.27,14313.99,7996.01,27613.49,...,9.0,15.0,17.0,20.0,28.0,20.0,26.0,23233.862083,17544.250744,70.833333
118427,142293,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5540.26,5530.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5557.343333,31.688589,0.166667
118428,142294,154954.39,118143.27,177612.09,167499.88,67058.13,80188.90,49690.89,58346.21,64483.23,...,78.0,81.0,98.0,89.0,59.0,110.0,97.0,92522.760833,40884.579699,99.000000
118429,142295,44802.68,34973.68,26061.68,23146.68,109953.68,98575.68,96178.18,88229.55,81233.55,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,59856.048750,30704.709009,1.791667


Pick only 4 relevant columns for model training. Save the data for future retrieval.

In [87]:
df_sme_2018_2019 = df_sme_2018_2019[['AccountID', 'BalanceMean', 'BalanceSD', 'Transactions']]

In [88]:
df_sme_2018_2019.to_csv('sme_2018_2019.csv', index = False)

## 1.3 Preparing Full Dataset

In this section, the 2018 and 2019 data is combined with the SME data to make the full data set.

### Joining Tables

In [89]:
df_sme_info.shape

(137159, 22)

In [90]:
df_sme_2018_2019.shape

(116607, 4)

Inner join is used to discard accounts that are missing on either table.

In [91]:
df_sme = df_sme_info.merge(df_sme_2018_2019, how = 'inner', on = 'AccountID')

### Exploring Data

In [92]:
df_sme.shape

(112618, 25)

In [93]:
df_sme.head()

,AccountID,Years,Islamic,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,Industry_Logistics,Industry_Manufacturing,...,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor,ProductType_CA,ProductType_SA,BalanceMean,BalanceSD,Transactions
0,000001,19.0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,27061.180000,15117.744298,44.625000
1,000002,4.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1.041667,2.074256,0.208333
2,000004,12.0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,62.964167,155.826156,3.791667
3,000005,5.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,328008.590000,111380.697708,138.750000
4,000006,15.0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,3843.160417,5122.731455,1.166667


In [94]:
df_sme.isnull().sum()

AccountID                   0
Years                       0
Islamic                     0
Industry_Agriculture        0
Industry_Construction       0
Industry_EducationHealth    0
Industry_Finance            0
Industry_Household          0
Industry_Logistics          0
Industry_Manufacturing      0
Industry_Mining             0
Industry_Others             0
Industry_RealEstate         0
Industry_Trade              0
Industry_Utilities          0
ConstType_Partnership       0
ConstType_PrivateLtd        0
ConstType_PublicLtd         0
ConstType_SchoolSociety     0
ConstType_SoleProprietor    0
ProductType_CA              0
ProductType_SA              0
BalanceMean                 0
BalanceSD                   0
Transactions                0
dtype: int64

In [95]:
df_sme.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112618 entries, 0 to 112617
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   AccountID                 112618 non-null  object 
 1   Years                     112618 non-null  float64
 2   Islamic                   112618 non-null  int64  
 3   Industry_Agriculture      112618 non-null  uint8  
 4   Industry_Construction     112618 non-null  uint8  
 5   Industry_EducationHealth  112618 non-null  uint8  
 6   Industry_Finance          112618 non-null  uint8  
 7   Industry_Household        112618 non-null  uint8  
 8   Industry_Logistics        112618 non-null  uint8  
 9   Industry_Manufacturing    112618 non-null  uint8  
 10  Industry_Mining           112618 non-null  uint8  
 11  Industry_Others           112618 non-null  uint8  
 12  Industry_RealEstate       112618 non-null  uint8  
 13  Industry_Trade            112618 non-null  u

In [96]:
df_sme.columns

Index(['AccountID', 'Years', 'Islamic', 'Industry_Agriculture',
       'Industry_Construction', 'Industry_EducationHealth', 'Industry_Finance',
       'Industry_Household', 'Industry_Logistics', 'Industry_Manufacturing',
       'Industry_Mining', 'Industry_Others', 'Industry_RealEstate',
       'Industry_Trade', 'Industry_Utilities', 'ConstType_Partnership',
       'ConstType_PrivateLtd', 'ConstType_PublicLtd',
       'ConstType_SchoolSociety', 'ConstType_SoleProprietor', 'ProductType_CA',
       'ProductType_SA', 'BalanceMean', 'BalanceSD', 'Transactions'],
      dtype='object')

In [97]:
for colname in df_sme.columns:
    print(f'Checking values for column "{colname}"')
    print(df_sme[colname].value_counts())
    print()

Checking values for column "AccountID"
066566    2
001630    2
113535    2
053740    1
111520    1
         ..
007759    1
072559    1
050328    1
092842    1
104609    1
Name: AccountID, Length: 112615, dtype: int64

Checking values for column "Years"
3.0      18008
4.0      14663
5.0      12784
2.0      12772
6.0       7093
         ...  
81.0         1
96.0         1
76.0         1
120.0        1
83.0         1
Name: Years, Length: 92, dtype: int64

Checking values for column "Islamic"
0    91040
1    21578
Name: Islamic, dtype: int64

Checking values for column "Industry_Agriculture"
0    109979
1      2639
Name: Industry_Agriculture, dtype: int64

Checking values for column "Industry_Construction"
0    97750
1    14868
Name: Industry_Construction, dtype: int64

Checking values for column "Industry_EducationHealth"
0    101792
1     10826
Name: Industry_EducationHealth, dtype: int64

Checking values for column "Industry_Finance"
0    89718
1    22900
Name: Industry_Finance, dtype: 

The product type columns are dropped as they are found to contain only one value (after previous data cleaning).

In [98]:
df_sme.drop(['ProductType_CA'], axis = 1, inplace = True)
df_sme.drop(['ProductType_SA'], axis = 1, inplace = True)

In [99]:
df_sme.head()

,AccountID,Years,Islamic,Industry_Agriculture,Industry_Construction,Industry_EducationHealth,Industry_Finance,Industry_Household,Industry_Logistics,Industry_Manufacturing,...,Industry_Trade,Industry_Utilities,ConstType_Partnership,ConstType_PrivateLtd,ConstType_PublicLtd,ConstType_SchoolSociety,ConstType_SoleProprietor,BalanceMean,BalanceSD,Transactions
0,000001,19.0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,27061.180000,15117.744298,44.625000
1,000002,4.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1.041667,2.074256,0.208333
2,000004,12.0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,62.964167,155.826156,3.791667
3,000005,5.0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,328008.590000,111380.697708,138.750000
4,000006,15.0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,3843.160417,5122.731455,1.166667


Save the data for future retrieval.

In [100]:
df_sme.to_csv('sme.csv', index = False)

To be continued...